In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class Value:
    
    def __init__(self, data, _children=(), _op=(), label=None):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    def __repr__(self):
        return f"Value(data={self.data}, label={self.label})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
            
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supports int/float for now"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        
        out._backward = _backward
        return out
    
    def __radd__(self, other): # other + self
        return self + other

    def __rmul__(self, other): # other * self
        return self * other
    
    def __truediv__(self, other): # self / other
        return self * other**-1
    
    def __neg__(self): # -self
        return self * -1
    
    def __sub__(self, other): # self - other
        return self + (-other)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        
        out._backward = _backward
        return out 
    
    def backward(self):
        def topo_sort(node):
            topo = []
            visited = set()
            def build_topo(v):
                if v not in visited:
                    visited.add(v)
                    for child in v._prev:
                        build_topo(child)
                    topo.append(v)
            build_topo(node)
            return topo
        
        self.grad = 1.0
        for node in reversed(topo_sort(self)):
            node._backward()

In [3]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi,xi in zip(self.w, x, strict=True)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b] # list

class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [4]:
def mse(targets, preds):
    return sum((p - t)**2 for t,p in zip(targets, preds))

In [5]:
# 3 dim dataset
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # targets

In [6]:
# THE TRAINING LOOP

In [7]:
# input
x = [2.0, 3.0, 4.0]

# model
n = MLP(3, [4,4,1])

# learning rate
step_size = 0.1

# gradient descent - n steps
for k in range(20):
    
    # forward pass
    ypreds = [n(x) for x in xs]
    loss = mse(ys, ypreds)
    
    # reset the grads - don't accumulate over epochs
    for p in n.parameters():
        p.grad = 0.0

    # backward pass
    loss.backward()
    
    # update
    for p in n.parameters():
        p.data += -step_size * p.grad

    # print loss
    print(k, loss.data)

0 4.5989613064588895
1 0.3584984086453715
2 0.043597093334512685
3 0.038980250195659635
4 0.035286792678
5 0.03225165324586955
6 0.029706271576024575
7 0.027537129811845553
8 0.025664370896170186
9 0.02402986077885475
10 0.022590095268598696
11 0.021311769692741313
12 0.02016889638279868
13 0.019140864421261918
14 0.018211095651301357
15 0.01736609061112408
16 0.016594736758564276
17 0.01588779750356661
18 0.01523752857996599
19 0.014637385813063277


In [8]:
# lets check the predictions
ypreds = [n(x) for x in xs]
ypreds

[Value(data=0.9542844486464731, label=None),
 Value(data=-0.9364568038027772, label=None),
 Value(data=-0.9405401213509124, label=None),
 Value(data=0.9335269000934927, label=None)]

In [9]:
ys

[1.0, -1.0, -1.0, 1.0]

In [10]:
# pretty close yeah !!!!